## ASTR 597A Homework 7
Eric Bellm

Due Feb 21, 2023

Your name: Tom Wagg
    
Your collaborators:

For this homework we'll need another external package, [sncosmo](https://sncosmo.readthedocs.io/).  Install it and then restart the kernel:

In [1]:
!pip install --user sncosmo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.5/656.5 kB 9.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.4/443.4 kB 19.6 MB/s eta 0:00:00


In [1]:
import sncosmo

In [14]:
import time
from IPython.display import Image

import numpy as np
import matplotlib.pyplot as plt

from lsst.rsp import get_tap_service

from astropy.cosmology import FlatLambdaCDM

from astropy.stats import sigma_clipped_stats

%config InlineBackend.figure_format = "retina"

import astropy.units as u
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
service = get_tap_service()
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [4]:
plt.style.use('tableau-colorblind10')

plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

## Exercise 1: Searching for Supernovae

Without using the truth table information, identify a sample of 40 or more candidate Ia supernovae.

The 07a tutorial notebook may be helpful.

In [5]:
redshift_min = 0.1
redshift_max = 0.3

snia_peak_mag = -19.0
snia_peak_mag_range = 0.5

snia_peak_mr_min = cosmo.distmod(redshift_min).value + snia_peak_mag - snia_peak_mag_range
snia_peak_mr_max = cosmo.distmod(redshift_max).value + snia_peak_mag + snia_peak_mag_range
print('The minimum and maximum apparent r-band magnitudes '
      'to use in the TAP query are %5.2f and %5.2f mag.' %
      (snia_peak_mr_min, snia_peak_mr_max))

snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0

snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5

snia_nDiaSources_min = 15
snia_nDiaSources_max = 100

snia_duration_min = 50
snia_duration_max = 300

The minimum and maximum apparent r-band magnitudes to use in the TAP query are 18.82 and 22.46 mag.


In [9]:
%%time

results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagAmp "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=2500)

DiaObjs = results.to_table()
del results

CPU times: user 96.7 ms, sys: 3.64 ms, total: 100 ms
Wall time: 1.8 s


/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [10]:
%%time

DiaObjs['duration'] = np.zeros(len(DiaObjs), dtype='float')

for j,DiaObjId in enumerate(DiaObjs['diaObjectId']):
    results = service.search("SELECT diaObjectId, midPointTai "
                             "FROM dp02_dc2_catalogs.DiaSource "
                             "WHERE diaObjectId = "+str(DiaObjId))
    results = results.to_table()
    DiaObjs['duration'][j] = np.max(results['midPointTai']) - np.min(results['midPointTai'])
    del results

CPU times: user 20.3 s, sys: 858 ms, total: 21.1 s
Wall time: 4min 23s


In [12]:
tx = np.where((DiaObjs['duration'] > snia_duration_min)
                 & (DiaObjs['duration'] < snia_duration_max))[0]
print(len(tx))

112


In [13]:
sn_ids = DiaObjs.to_pandas().loc[tx]["diaObjectId"].values
np.save("potential_sn_ids", sn_ids)

In [6]:
sn_ids = np.load("potential_sn_ids.npy")

## Exercise 2: Ia Lightcurve fitting.

Using `sncosmo`, fit a [SALT2](https://ui.adsabs.harvard.edu/abs/2007A%26A...466...11G/abstract) model to each of the candidate Ia SNe you identified in Exercise 1.  

The fit to each lightcurve will yield parameters `z` (redshift), `t0` (time of peak brightness), `x0` (amplitude), `x1` (stretch), and `c` (color).  Save them and their errors. 

Note that the fit may fail for some of your lightcurves; that's okay.

In [16]:
model = sncosmo.Model(source='salt2')

In [18]:
%%time
converged = np.repeat(True, len(tx))
parameters = np.zeros((len(tx), 5))
errors = np.zeros((len(tx), 5))

for j in range(len(tx)):
    results = service.search("SELECT midPointTai, filterName AS band, psFlux AS flux, psFluxErr as fluxerr "
                             "FROM dp02_dc2_catalogs.DiaSource "
                             "WHERE diaObjectId = "+str(DiaObjs['diaObjectId'][tx[j]]))
    results = results.to_table()
    results.rename_column("midPointTai", "time")

    for i in range(len(results)):
        results[i]["band"] = "lsst" + results[i]["band"]

    results["zp"] = np.ones(len(results)) * -25.0
    bands = 'ugrizy'
    zps = [-27.03, -28.38, -28.16, -27.85, -27.46, -26.68]
    for band, zp in zip(bands, zps):
        results["zp"][results["band"] == f'lsst{band}'] = zp
    
    results["zpsys"] = np.repeat("ab", len(results))

    try:
        lc = sncosmo.fit_lc(results, model, ['z', 't0', 'x0', 'x1', 'c'],
                            bounds={'z':(0.1, 0.3)})
    except:
        converged[j] = False
        continue
        
    parameters[j] = lc[0]["parameters"]
    errors[j] = [lc[0]["errors"][param_name] for param_name in lc[0]["param_names"]]

CPU times: user 16.8 s, sys: 73.1 ms, total: 16.9 s
Wall time: 27.7 s


## Exercise 3: My First Hubble Diagram

Now we will use our SALT2 fit values to create a rudimentary Hubble diagram.  We will plot the distance modulus $\mu$ derived from the SALT fits as a function of redshift.

<div class="alert alert-block alert-warning">
    <b>Warning:</b> This analaysis approach is intended to be illustrative, and doesn't reflect the current state of the art for Ia cosmology.  The interested reader is encouraged to consult the literature (e.g., <a href="https://ui.adsabs.harvard.edu/abs/2022ApJ...938..111B/abstract">Brout+22</a>, <a href="https://ui.adsabs.harvard.edu/abs/2022arXiv221107657D/abstract">Dhawan+22</a>, <a href="https://ui.adsabs.harvard.edu/abs/2022ApJ...934...96S/abstract">Sanchez+22</a>, and references therein) to learn more.

</div>

The [Tripp Law](https://ui.adsabs.harvard.edu/abs/1998A%26A...331..815T/abstract) can be [written](https://ui.adsabs.harvard.edu/abs/2011ApJ...740...72M/abstract) in terms of the SALT2 model parameters:

$\mu = -2.5 \log_{10}{x_0} + \alpha x_1 - \beta c - M_0$

The parameters $\alpha$, $\beta$, and $M_0$ are typically fit along with the cosmological model.  For simplicity, we will adopt the DC2 values $\alpha = 0.137$, $\beta=3.21$ (see Table 1 of [Sanchez+22](https://ui.adsabs.harvard.edu/abs/2022ApJ...934...96S/abstract)).

### Exercise 3a:

Using the above formula, plot the distance modulus versus redshift for our Ia supernovae.  Overplot the theoretical luminosity distance vs. redshift curve for a fiducial cosmology.  Use "chi by eye" to adjust $M_0$ until the points overlap reasonably with the model.

In [19]:
quality = (DiaObjs[tx]["duration"] < 100) & (errors[:, 2] < 1.5) & (errors[:, 3] < 0.5)\
    & ~(np.isclose(parameters[:, 0], 0.1)) & ~(np.isclose(parameters[:, 0], 0.3))

In [20]:
z = parameters[:, 0][converged & quality]
x0 = parameters[:, 2][converged & quality]
x1 = parameters[:, 3][converged & quality]

In [23]:
def mu(M0):
    alpha = 0.137
    beta = 3.21
    return -2.5 * np.log10(x0) + (alpha * x1) - beta - M0

def dist_from_modulus(mu):
    return 10**(1 + (mu / 5))

def modulus_from_dist(dist):
    return (np.log10(dist.to(u.pc).value) - 1) * 5

In [24]:
lum_dist = cosmo.luminosity_distance(z)
dist_mod = (np.log10(lum_dist.to(u.pc).value) - 1) * 5
order = np.argsort(lum_dist)

In [43]:
residuals = dist_mod - mu(-100)
np.std(residuals)

0.8050125312681311

In [28]:
def plot_dist_mod(M0):
    fig, ax = plt.subplots()

    ax.scatter(z, mu(M0), label="SNIa Candidates")
    ax.plot(z[order], dist_mod[order], label="Theoretical distance-redshift relation", color="tab:purple", lw=3)

    
    
    ax.legend()

    ax.set_xlabel(r"Redshift, $z$")
    ax.set_ylabel(r"Distance Modulus")

    plt.show()

    residuals = dist_mod - mu(M0)
    return np.std(residuals)

In [31]:
interact(plot_dist_mod, M0=(-92, -90, 0.1))

interactive(children=(FloatSlider(value=-91.0, description='M0', max=-90.0, min=-92.0), Output()), _dom_classe…

<function __main__.plot_dist_mod(M0)>

## Exercise 3b:

Compute the standard deviation of the residuals between the fit $\mu$ values and the luminosity distance expected at that redshift.  How does that compare to the simulated "intrinsic scatter" of 0.15 mag in the DC2 simulations (Table 1 of [Sanchez+22](https://ui.adsabs.harvard.edu/abs/2022ApJ...934...96S/abstract))? 

What steps could you imagine taking to reduce the scatter in your Hubble diagram?

In [44]:
best_M = -91.5
residuals = dist_mod - mu(best_M)

In [49]:
np.std(residuals)

0.8050125312681311

This is clearly much higher than the 0.15mag in the simulations. However we have taken very rudimentary metric for what constitutes a SNIa and so it is likely that some of the sample is not supposed to be there. We should go through and inspect each of the lightcurves and see which ones actually seem like they are true SNIa. This should remove many of the outliers from the plot (e.g. the one around z=0.225). I would probably need a larger sample as well as that point but that would just require running all of this for a tad longer.